In [6]:
from goodreads import client
import requests
import re
import urllib2
from bs4 import BeautifulSoup
import sys
import csv
import json
from collections import defaultdict
import xml.etree.ElementTree as ET 

# Scraping reviews

In [7]:
#use your own api_key
api_key = "Je8sMDXDmPqMd1MPknxdw" 
api_secret_key= "FqboDUT1Ym4Ra5WHrCfjaCBsy5cVhJVtfqrYwdqM"

In [8]:
gc=client.GoodreadsClient(api_key,api_secret_key)

In [70]:
author_rev_1=dict() #dictionary
for i in range(1,10):
    bookname= gc.book(i)
    res1=requests.get("https://www.goodreads.com/book/show.json",params={"key":api_key,
                                                                         "id":i})
    book=re.findall('(?<=src=\")(.*?)(?=\")',res1.json()['reviews_widget'])
    quote_page =str(book[0])
    page = urllib2.urlopen(quote_page)
    soup = BeautifulSoup(page,"html.parser")
    
    reviewcontainers = soup.find('div',attrs={"class": "gr_reviews_container"}).findAll('div', attrs={'class': 'gr_review_container'})
    if len(reviewcontainers)==10:
        review=[]
        for i in reviewcontainers:
            link= re.findall('(?<=href=\")(.*?)(?=\")',
                             str(i.find('div', attrs={'class':'gr_review_text'}).find('a', attrs={'class':'gr_more_link'})))[0]
            page2 = urllib2.urlopen(link)
            nextlink=BeautifulSoup(page2,"html.parser")
            review.append(nextlink.find('div',attrs={'class':'reviewText mediumText description readable'}).text.strip())
        author_rev_1[str(bookname)]=review

# Importing to a Json File

In [71]:
with open('first10book.json', 'w') as fp:
    json.dump(author_rev_1, fp)

NameError: name 'author_rev_1' is not defined

# Importing a list of bookID from CSV file

In [8]:
csv_reader = csv.reader(open("books.csv", 'rb'))
k10book= list(csv_reader)
bookid10k=[k10book[i][2] for i in range(0,10000)][1:]

# Sample book reviews from 20 books

In [12]:
sample_20 = bookid10k[:20]

In [13]:
booktitle_review_sample20=dict() #dictionary
for i in sample_20:
    bookname= gc.book(i)
    res1=requests.get("https://www.goodreads.com/book/show.json",params={"key":api_key,
                                                                         "id":i})
    book=re.findall('(?<=src=\")(.*?)(?=\")',res1.json()['reviews_widget'])
    quote_page =str(book[0])
    page = urllib2.urlopen(quote_page)
    soup = BeautifulSoup(page,"html.parser")
    
    reviewcontainers = soup.find('div',attrs={"class": "gr_reviews_container"}).findAll('div', attrs={'class': 'gr_review_container'})
    if len(reviewcontainers)==10:
        review=[]
        for i in reviewcontainers:
            link= re.findall('(?<=href=\")(.*?)(?=\")',
                             str(i.find('div', attrs={'class':'gr_review_text'}).find('a', attrs={'class':'gr_more_link'})))[0]
            page2 = urllib2.urlopen(link)
            nextlink=BeautifulSoup(page2,"html.parser")
            review.append(nextlink.find('div',attrs={'class':'reviewText mediumText description readable'}).text.strip())
        booktitle_review_sample20[str(bookname)]=review

In [17]:
with open('first20booksample.json', 'w') as fp:
    json.dump(booktitle_review_sample20, fp)

# Most Optimal version

In [74]:
def Scrape_review(start,end):
    booktitle_review=dict() #dictionary
    for i in range(start,end):
        try:
            res1=requests.get("https://www.goodreads.com/book/show.json",params={"key":api_key,
                                                                                 "id":i})
            book=re.findall('(?<=src=\")(.*?)(?=\")',res1.json()['reviews_widget'])
            quote_page =str(book[0])
            page = urllib2.urlopen(quote_page)
            soup = BeautifulSoup(page,"html.parser")
            reviewcontainers = soup.find('div',attrs={"class": "gr_reviews_container"}).findAll(
                'div', attrs={'class': 'gr_review_container'})
            
            if len(reviewcontainers)==10:
                review=[]
                counter = 0
                for i in reviewcontainers:
                    link= re.findall('(?<=href=\")(.*?)(?=\")',
                                     str(i.find('div', attrs={'class':'gr_review_text'}).find('a', attrs={'class':'gr_more_link'})))[0]
                    page2 = urllib2.urlopen(link)
                    nextlink=BeautifulSoup(page2,"html.parser")
                    review.append(nextlink.find('div',attrs={'class':'reviewText mediumText description readable'}).text.strip())
                    if counter == 0:
                        bookname = str(nextlink.find('a', attrs = {'class':'bookTitle'}).text.strip())
                        counter += 1
                booktitle_review[bookname]=review
        except Exception:
            None
    return booktitle_review

In [9]:
def Scrape_review(start,end):
    resultdict = defaultdict(dict)
 
    #booktitle_review=dict() #dictionary
    for i in range(start,end):
        booktitle  = dict() 
        try:
            res2 = requests.get("https://www.goodreads.com/book/show.xml", params={"key":api_key,"id":i})
            root  = ET.fromstring(res2.content)  

            review_count = root.find('book').find('text_reviews_count').text;
            author = root.find('book').find('authors').find('author').find('name').text; 
            title = root.find('book').find('title').text; 
            description = root.find('book').find('description').text 
            average_rating = root.find('book').find('average_rating').text  
            booktitle['review_count'] = review_count 
            booktitle['author'] = author  
            booktitle['description'] = description 
            booktitle['average_rating'] = average_rating 
          

            book=re.findall('(?<=src=\")(.*?)(?=\")',root.find('book').find('reviews_widget').text)
            quote_page =str(book[0])
    
            page = urllib2.urlopen(quote_page)
            soup = BeautifulSoup(page,"html.parser")
            reviewcontainers = soup.find('div',attrs={"class": "gr_reviews_container"}).findAll(
                'div', attrs={'class': 'gr_review_container'})
           
            if len(reviewcontainers)==10: 
                totalreview = [] 
                counter = 0
                for i in reviewcontainers:
                    review=[] 
                    link= re.findall('(?<=href=\")(.*?)(?=\")',
                                     str(i.find('div', attrs={'class':'gr_review_text'}).find('a', attrs={'class':'gr_more_link'})))[0]
                    page2 = urllib2.urlopen(link)
                    nextlink=BeautifulSoup(page2,"html.parser")
                    review.append(nextlink.find('div',attrs={'class':'reviewText mediumText description readable'}).text.strip())
                    if counter == 0:
                        bookname = str(nextlink.find('a', attrs = {'class':'bookTitle'}).text.strip())
                        counter += 1
                    totalreview.append(review) 
            booktitle['review'] = totalreview 

            resultdict[title] = booktitle 

   
        except Exception:
            print("exception")

    return resultdict


In [10]:
booktitle_reveow_sample400 = Scrape_review(400,499)
with open('400-499.json', 'w') as fp:
    json.dump(booktitle_reveow_sample400, fp)

exception
exception
exception
exception
exception
exception
exception
exception
exception
